<a href="https://colab.research.google.com/github/khalidpark/deeplearning_whitepaper/blob/main/deep_learning_whitepaper_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 데이터를 불러옵니다. 
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

# 정규화를 위한 함수 호출
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
print(x_train[:10])


57344/57026 [==============================] - 0s 0us/step
[[-0.27224633 -0.48361547 -0.43576161 -0.25683275 -0.1652266  -0.1764426
   0.81306188  0.1166983  -0.62624905 -0.59517003  1.14850044  0.44807713
   0.8252202 ]
 [-0.40342651  2.99178419 -1.33391162 -0.25683275 -1.21518188  1.89434613
  -1.91036058  1.24758524 -0.85646254 -0.34843254 -1.71818909  0.43190599
  -1.32920239]
 [ 0.1249402  -0.48361547  1.0283258  -0.25683275  0.62864202 -1.82968811
   1.11048828 -1.18743907  1.67588577  1.5652875   0.78447637  0.22061726
  -1.30850006]
 [-0.40149354 -0.48361547 -0.86940196 -0.25683275 -0.3615597  -0.3245576
  -1.23667187  1.10717989 -0.51114231 -1.094663    0.78447637  0.44807713
  -0.65292624]
 [-0.0056343  -0.48361547  1.0283258  -0.25683275  1.32861221  0.15364225
   0.69480801 -0.57857203  1.67588577  1.5652875   0.78447637  0.3898823
   0.26349695]
 [-0.37502238 -0.48361547 -0.54747912 -0.25683275 -0.54935658 -0.78865126
   0.18954148  0.48371503 -0.51114231 -0.71552978  0.51

# 배치 사이즈

In [2]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# 재현성을 위해 랜덤시드를 생성합니다
#seed = 7
numpy.random.seed(1100)

# 데이터셋을 불러옵니다.
url ="https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

dataset = pd.read_csv(url, header=None).values

# 불러온 데이터셋을 X와 Y로 나눕니다
X = dataset[:,0:8]
Y = dataset[:,8]

# 모델을 만들기 위한 함수를 만듭니다 (KerasClassifier 요구 사항)
def create_model():
    # 모델 제작
    model = Sequential()
    model.add(Dense(100, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # 모델 컴파일링
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# keras.wrapper를 활용하여 분류기를 만듭니다
model = KerasClassifier(build_fn=create_model, verbose=0)

# GridSearch
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [30]
param_grid = dict(batch_size=batch_size)

# GridSearch CV를 만듭니다
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# 최적의 결과값을 낸 파라미터를 출력합니다
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")


Best: 0.5794499576091766 using {'batch_size': 80}
Means: 0.559723287820816, Stdev: 0.07163243539060642 with: {'batch_size': 10}
Means: 0.5572786867618561, Stdev: 0.039976298252155186 with: {'batch_size': 20}
Means: 0.5090909123420715, Stdev: 0.06601747986798673 with: {'batch_size': 40}
Means: 0.5469654679298401, Stdev: 0.04902355863504818 with: {'batch_size': 60}
Means: 0.5794499576091766, Stdev: 0.10910108262483108 with: {'batch_size': 80}
Means: 0.5286308526992798, Stdev: 0.08944662826467799 with: {'batch_size': 100}


# Keras Tuner 를 사용한 파라미터 튜닝

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten
import IPython


In [4]:
!pip install -U keras-tuner
import kerastuner as kt


     |████████████████████████████████| 71kB 2.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=dc2bd12a003a4cf9d49351a3a81a55357242ab6e34a89956152cabeb78e591c0
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=c403de63f097f76643e98038eaffaa9bc6130f853def69e93b85290ce51a5c18
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [5]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()


4423680/4422102 [==============================] - 0s 0us/step


In [6]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0


모델 구조 제작

In [7]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(Flatten(input_shape=(28, 28)))
  
  # 첫 번째 Dense layer에서 노드 수를 조정(32-512)합니다.
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(Dense(units = hp_units, activation = 'relu'))
  model.add(Dense(10))

  # Optimizer의 학습률(learning rate)을 조정[0.01, 0.001, 0.0001]합니다. 
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model


튜너를 인스턴스화하고 하이퍼 튜닝을 실행

In [8]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10, # 일반적으로 10, 빠른 구현을 위해서 숫자를 줄였음.
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')


callback 정의

In [9]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)


In [10]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 {best_hps.get('units')} 입니다.
최적의 학습 속도는 {best_hps.get('learning_rate')} 입니다.
""")


Trial 30 Complete [00h 00m 41s]
val_accuracy: 0.8741000294685364

Best val_accuracy So Far: 0.8902999758720398
Total elapsed time: 00h 09m 11s
INFO:tensorflow:Oracle triggered exit

하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 512 입니다.
최적의 학습 속도는 0.001 입니다.



In [11]:
# 최적의 하이퍼 파라미터를 사용하여 모델을 구축하고 데이터에 대해 교육
model = tuner.hypermodel.build(best_hps)
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))


Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.5868 - accuracy: 0.7932 - val_loss: 0.4220 - val_accuracy: 0.8459
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3675 - accuracy: 0.8656 - val_loss: 0.3686 - val_accuracy: 0.8665
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3291 - accuracy: 0.8781 - val_loss: 0.3480 - val_accuracy: 0.8740
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2994 - accuracy: 0.8884 - val_loss: 0.3322 - val_accuracy: 0.8788
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2788 - accuracy: 0.8951 - val_loss: 0.3256 - val_accuracy: 0.8819
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2649 - accuracy: 0.9010 - val_loss: 0.3330 - val_accuracy: 0.8799
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2528 - accuracy: 0.9057 - val_loss: 0.3300 - val_accuracy: